# i. Perkenalan

Nama : Akbar Fitriawan  
Batch : HCK-15

Tentang dataset:


Objective:


---

# ii. Import Libraries

In [ ]:
# library dataframe
import pandas as pd
# library olah data numerical
import numpy as np
# library statistik
from scipy.stats import pearsonr, kendalltau, spearmanr

# Data visualisasi
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# tensorflow 
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Dense, Flatten


# libraries Data Preprocessing
from sklearn.model_selection import train_test_split # splittingdata
from feature_engine.outliers import Winsorizer # outlier
from sklearn.preprocessing import StandardScaler, MinMaxScaler,RobustScaler # scalling
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder # encoder

from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

# Library handling outlier
from feature_engine.outliers import Winsorizer

# Library to suppress warnings or deprecation notes
import warnings
warnings.filterwarnings('ignore')

---

# iii. Data Loading

In [ ]:
df = pd.read_csv('')
df

In [ ]:
print('Jumlah Baris: ',df.shape[0])
print('Jumlah Kolom: ',df.shape[1])

In [ ]:
# Data 5 teratas 
df.head()

In [ ]:
# Data 5 Terbawah
df.tail()

In [ ]:
# Data info
df.info()

Insight:
- Dataset memiliki `1470` jumlah baris dan `22` jumlah kolom
- Data type int64(13), object(9)
- Tidak ada indikasi missing values
- EmployeeID tidak digunakan karena tidak ada informasi yang dapat diambil

In [ ]:
# Getting Data Info 
df.describe(include=np.number)

In [ ]:
df.describe(include='object')

---

# iv. Exploratory Data Analysis (EDA)

In [ ]:
# copy dataset
data_analyz = df.copy()

In [ ]:
# Distribution Data attration employee
plt.figure(figsize=(8,6))
data_analyz.Attrition.value_counts().plot(kind='pie', autopct='%1.1f%%', startangle=140)
plt.title('Total Count Attrion Employee')
plt.show()

**Conclusions**

---

# v. Feature Engineering

In [ ]:
# dump dataset agar tidak mengulang
data = df.copy()

In [ ]:
# cek duplicated data
data.duplicated().sum()

In [ ]:
# checking distribution selected feature
to_Plot = ['Age', 'DistanceFromHome', 'MonthlyIncome',
       'NumCompaniesWorked', 'PercentSalaryHike',
       'TotalWorkingYears', 'YearsAtCompany', 'YearsInCurrentRole',
       'YearsSinceLastPromotion', 'YearsWithCurrentManager','Attrition']
plt.figure()
sns.pairplot(data[to_Plot],hue='Attrition')  
#Taking hue 
plt.show()

## Spliting Data

In [ ]:
# split data feature dan target
X = data.drop('Attrition', axis=1)
y = data['Attrition']

In [ ]:
# Splitting between Train-Set, Val-Set, and Test-Set

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.15, random_state=42)

print('Train Size : ', X_train.shape)
print('Val Size   : ', X_val.shape)
print('Test Size  : ', X_test.shape)

## Checking Cardinality

In [ ]:
def tampilkan_nilai_unik(data):
    for column in data.columns:
        print(f"{column}: {data[column].unique()}")

In [ ]:
tampilkan_nilai_unik(X_train[cat])

In [ ]:
data.nunique().to_frame().reset_index().rename(columns={'index':'Column',0:'Unique_values'})

## Handling Missing Value

In [ ]:
# Check Missing Values on X_train

X_train.isnull().sum()

In [ ]:
# Check Missing Values on X_val

X_val.isnull().sum()

In [ ]:
# Check Missing Values on X_test

X_test.isnull().sum()

In [ ]:
# Checking outlier

# plot figure 
fig, axes = plt.subplots(4, 3, figsize=(15, 10))
axes = axes.flatten()
# membuat boxplot
for i, column in enumerate(num):
    axes[i].boxplot(data[column])
    axes[i].set_title(column)
    
# hapus plot kosong
for j in range(len(num), len(axes)):
    axes[j].axis('off')

plt.tight_layout()

plt.show()

Dari hasil boxplot yang memiliki outlier adalah `MonthlyIncome`, `TotalWorkingYears`, `YearsAtCompany`, `YearsCurrentRole`,`YearsSinceLastPromotion`, dan `YearsWithCurrentManager`. adapun kolom lain yang memiliki outlier tapi bukankah fakta nyata jadi saya abaikan

Seperti penjelasan di atas adanya outlier sehingga harus di atasi, dalam hal ini saya menggunakan metode Winsorization.

Cara melakukannya adalah memfilter ditribusi normal dan tidak dengan menggunakan motode statistika skewnes dan menset threshold.
- Threshold untuk Distribusi Normal: 
Biasanya, nilai skewness yang mendekati nol (`misalnya, antara -0.5 hingga 0.5`) menunjukkan bahwa distribusi data cenderung normal. 
- Threshold untuk Distribusi Skewed:
Distribusi data dikatakan skewed jika nilai skewnessnya melebihi nilai ambang tertentu. Secara umum, `nilai skewness di atas 0.5 atau di bawah -0.5` sering digunakan sebagai indikator bahwa distribusi data tidak simetris dan cenderung skewed. 


In [ ]:
# Filter Numerical Columns into Variable
# num_cols = X_train.select_dtypes(include=np.number).columns.to_list()
num_cols = ['Age', 'DistanceFromHome','MonthlyIncome','PercentSalaryHike','TotalWorkingYears','YearsAtCompany','YearsInCurrentRole','YearsSinceLastPromotion','YearsWithCurrentManager']

# Making data and columns for normal distribution
dataNum_normal = []
listCols_normal = []

# Making data and columns for skewed distribution
dataNum_skewed = []
listCols_skewed = []

# For loop in every numerical column to filer the data distribution into either normal distributed or skewed columns
for cols in num_cols:
    skewness = X_train[cols].skew()
    
    # If the data normally distributed
    if skewness <= 0.5 and skewness >= -0.5:            
        listCols_normal.append(cols)
        dataNum_normal.append([cols, skewness])
        
    # Elif the data is skewed
    elif skewness < -0.5 or skewness > 0.5:
        listCols_skewed.append(cols)
        dataNum_skewed.append([cols, skewness])

In [ ]:
# Showing skewed columns
data_normal = pd.DataFrame(data=dataNum_normal, columns=['kolom_normal', 'skewness'])
data_normal

In [ ]:
# Showing skewed columns
data_skew = pd.DataFrame(data=dataNum_skewed, columns=['kolom_skewed', 'skewness'])
data_skew

In [ ]:
# # Input low variation columns into variable
# col_lowVarition = ['PerformanceRating']

# # Drop the low variation columns in skewed columns, and add it into normal distribution columns
# for kolom in col_lowVarition:
#     listCols_skewed.remove(kolom) #-> removing low variation columns in skewed columns
#     listCols_normal.append(kolom) #-> appending low variation columns in normal distributed columns

In [ ]:
# Capping Method for Normal Distribution  
winsorizer_normal_dist = Winsorizer(capping_method='gaussian',
                            tail='both',
                            fold=3,
                            variables=listCols_normal,
                            missing_values='ignore')

# Fit & Transforming X_train 
X_train_capped = winsorizer_normal_dist.fit_transform(X_train)

# Transforming X_test
X_val_capped = winsorizer_normal_dist.transform(X_val)

In [ ]:
winsorizer_skewed = Winsorizer(capping_method='iqr',
                            tail='both',
                            fold=1.5,
                            variables=listCols_skewed,
                            missing_values='ignore')

# Fit & Transforming X_train 
X_train_capped = winsorizer_skewed.fit_transform(X_train_capped)

# Transforming X_test
X_val_capped = winsorizer_skewed.transform(X_val_capped)

In [ ]:
# Plot Distribution Comparison
def outlier_handling_plot_comparison(df_before, df_after, variable):
    """
    This function is created to plot histograms and boxplots for a variable before and after outlier handling
    
    Parameters:
    - df_before (pandas.DataFrame): DataFrame before outlier handling
    - df_after (pandas.DataFrame): DataFrame after outlier handling
    - variable (str): The variable to plot
    
    Example: 
    num_cols = ['numeric_column']                                   <- Enter the numeric column
    for col in num_cols:                                            <- make a for loop in numeric column to access every columns
        plot_distribution_comparison(X_train, X_train_capped, col)  <- put X_train, X_train_capped, and columns in the variable

    """
    # Figure Size, and Super Title based on variable
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))                               
    fig.suptitle(f'{variable} - Distribution Before and After Outlier Handling')

    # Plot Histogram Before
    sns.histplot(df_before[variable], bins=30, ax=axes[0, 0], color='skyblue')
    axes[0, 0].set_title('Histogram Before')

    # Plot Boxplot Before
    sns.boxplot(y=df_before[variable], ax=axes[1, 0], color='lightgreen')
    axes[1, 0].set_title('Boxplot Before')

    # Plot Histogram After
    sns.histplot(df_after[variable], bins=30, ax=axes[0, 1], color='skyblue')
    axes[0, 1].set_title('Histogram After')

    # Plot Boxplot After
    sns.boxplot(y=df_after[variable], ax=axes[1, 1], color='lightgreen')
    axes[1, 1].set_title('Boxplot After')

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

In [ ]:
# Using the Function
for col in num_cols:
    outlier_handling_plot_comparison(X_train, X_train_capped, col)

## Spliting Data Numerik dan Kategorik

In [ ]:
# column numeric list
# num_columns = X_train_capped.select_dtypes(include=np.number).columns.tolist()
num_columns = [
    'Age',
    'DistanceFromHome',
    'MonthlyIncome',
    'NumCompaniesWorked',
    'PercentSalaryHike',
    'PerformanceRating',
    'TotalWorkingYears',
    'YearsAtCompany',
    'YearsInCurrentRole',
    'YearsSinceLastPromotion',
    'YearsWithCurrentManager'
]
# column categorical list
# cat_columns = X_train_capped.select_dtypes(include=['object']).columns.tolist()
cat_columns = [
    'BusinessTravel',
    'Department',
    'Education',
    'EducationField',
    'Gender',
    'JobRole',
    'JobSatisfaction',
    'MaritalStatus',
    'WorkLifeBalance'
]


print('Numerical List: ', num_columns)
print('Catgorical List: ', cat_columns)

In [ ]:
# override X_train with spliting list
X_train_num = X_train_capped[num_columns]
X_train_cat = X_train_capped[cat_columns]
# override X_test with spliting list
X_val_num = X_val_capped[num_columns]
X_val_cat = X_val_capped[cat_columns]

In [ ]:
print('Size X_train_num:',X_train_num.shape)
print('Size X_train_cat:',X_train_cat.shape)
print('\n')
print('Size X_test_num:',X_val_num.shape)
print('Size X_test_cat:',X_val_cat.shape)

## Feature Selection

Dalam melakukan Feature Selection yaitu dengan membagi kolom Numerical dan Categorical, Kemudian mencari korelasi yang berpengaruh terhadap target(label) dengan cara metode statitika. 

Mencari Korelasi:
- Untuk Categorical biasanya menggunakan `Kendall Tau's` dan Numerical menggunakan `pearsonr`(distribusi normal) serta `spearmanr` (distribusi skewed/not gaussian)
- Set treshold P-value 0,05.
    - jika nilai p-value kurang dari `< 0.05`, yang berarti terdapat korelasi yang signifikan antara variabel yang dianalisis
    - Jika nilai p-value sama dengan atau lebih besar dari `>= 0.05 atau > 0.05`, yang berarti tidak terdapat korelasi yang signifikan antara variabel yang dianalisis.

### Categories Feature

In [ ]:
# Finding the correlation between categorical columns and 'Attrition' using Kendall Tau's correlation

p_values = []
interpretation = []
cols = []
corr = []


for col in X_train_cat.columns:
  corr_coef, p_value = kendalltau(X_train_cat[col], y_train)

  p_values.append(p_value)
  cols.append(col)
  corr.append(corr_coef)

  if p_value < 0.05:
    interpretation.append('Significant Correlation')
  else :
    interpretation.append('No Significant Correlation')

cat_corr= pd.DataFrame({'Column Name':cols,
              'Correlation Coefficient' : corr,
              'P-value':p_values,
              'Interpretation': interpretation })

cat_corr

In [ ]:
# filter corelation categorical
cat_corr = cat_corr[cat_corr['Interpretation'] == "Significant Correlation"]

cat_corr

In [ ]:
list_cat = cat_corr['Column Name'].values.tolist()
list_cat

Jadi feature yang di pakai dari categorical column yaitu ('Department', 'JobRole', 'MaritalStatus')

### Numerical Feature

In [ ]:
# Finding the correlation between numerical columns and 'price' using pearsonr and spearmanr correlation

p_values = []
interpretation = []
cols = []
corr = []

for col in X_train_num.columns:
  if abs(X_train_num[col].skew()) < 0.5:                    #For Normally Distributed Columns
    corr_coef, p_value = pearsonr(X_train_num[col], y_train)

    p_values.append(p_value)
    cols.append(col)
    corr.append(corr_coef)

    if p_value < 0.05:
      interpretation.append('Significant Correlation')
    else :
      interpretation.append('No Significant Correlation')
  else:                                                     #For Non Normally Distributed Columns
    corr_coef, p_value = spearmanr(X_train_num[col], y_train)

    p_values.append(p_value)
    cols.append(col)
    corr.append(corr_coef)

    if p_value < 0.05:
      interpretation.append('Significant Correlation')
    else :
      interpretation.append('No Significant Correlation')

num_corr = pd.DataFrame({'Column Name':cols,
              'Correlation Coefficient' : corr,
              'P-value':p_values,
              'Interpretation': interpretation })

num_corr

In [ ]:
# filter corelation categorical
num_corr = num_corr[num_corr['Interpretation'] == "Significant Correlation"]
num_corr

In [ ]:
list_num  = num_corr['Column Name'].values.tolist()
list_num 

Dari korelasi Numerical column yaitu ('Age','MonthlyIncome','TotalWorkingYears','YearsAtCompany','YearsInCurrentRole','YearsWithCurrentManager')

<!-- ### Membuat variable baru Untuk Feature yang berkolerasi -->

In [ ]:
# print list correlation
print('column katgorik:', list_cat)
print('column numerik:', list_num)

In [ ]:
# append ke X feature yang berkorelasi
# X_train_cat = X_train_cat[list_cat]
# X_train_num = X_train_num[list_num]

# X_val_cat = X_val_cat[list_cat]
# X_val_num = X_val_num[list_num]

# X_test_cat = X_test[list_cat]
# X_test_num = X_test[list_num]

In [ ]:
# print('Size X_train_num:',X_train_num.shape)
# print('Size X_train_cat:',X_train_cat.shape)
# print('')
# print('Size X_val_num:',X_val_num.shape)
# print('Size X_val_cat:',X_val_cat.shape)
# print('')
# print('Size X_test_num:',X_test_num.shape)
# print('Size X_test_cat:',X_test_cat.shape)

In [ ]:
# Checking the numerical data
# X_train_num

In [ ]:
# Checking the categorical data
# X_train_cat

In [ ]:
# Checking the numerical data
# X_val_num

In [ ]:
# Checking the categorical data
# X_val_cat

In [ ]:
# Checking the numerical data
# X_test_num

In [ ]:
# Checking the numerical data
# X_test_cat

## Make Pipeline

In [ ]:
list_cat
list_num

In [ ]:
# Create A Pipeline

num_pipeline = make_pipeline(SimpleImputer(strategy='median'),
                             StandardScaler())

cat_pipeline = make_pipeline(OneHotEncoder())

final_pipeline = ColumnTransformer([
    ('pipe_num', num_pipeline, list_num),
    ('pipe_cat', cat_pipeline, list_cat)
])

In [ ]:
# Fit and Transform

X_train = final_pipeline.fit_transform(X_train)
X_val = final_pipeline.transform(X_val)
X_test = final_pipeline.transform(X_test)


In [ ]:
X_train.shape

In [ ]:
# checking distribution selected feature
plt.figure()
sns.pairplot(X_test_final)  
#Taking hue 
plt.show()

---

# vi. Modeling 

## A. Model Sequential

Notes of hyperparameter `loss` :
* Use `sparse_categorical_crossentropy` :
  + The target just a class index/number such as from `0` to `9` like in this case

* Use `categorical_crossentropy` :
  + The target is in form of one-hot vectors
  + To represent class `3` the target will be `[0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]`

### **1.a Model Definition**

In [ ]:
# Clear Session
seed = 20
tf.keras.backend.clear_session()
np.random.seed(seed)
tf.random.set_seed(seed)

ann = Sequential()
# ann.add() #input
ann.add(Dense(300, activation='relu', input_shape=(X_train.shape[1:]), kernel_initializer=tf.keras.initializers.HeNormal(seed))) #hidden
ann.add(Dense(100, activation='relu', kernel_initializer=tf.keras.initializers.HeNormal(seed))) #hidden
ann.add(Dense(300, activation='relu', kernel_initializer=tf.keras.initializers.HeNormal(seed))) #hidden
ann.add(Dense(100, activation='relu', kernel_initializer=tf.keras.initializers.HeNormal(seed))) #hidden
ann.add(Dense(1, activation='sigmoid', kernel_initializer=tf.keras.initializers.GlorotNormal(seed))) #output

# Compile
ann.compile(loss='binary_crossentropy', #sparse_categorical_crossentropy
                         optimizer='adam',
                         metrics=['accuracy'])
ann.summary()

In [ ]:
# Plot Layers

tf.keras.utils.plot_model(model_sequential_mnist, show_shapes=True)

### **2.a Model training**

In [ ]:
history_1= ann.fit(X_train, y_train,
        epochs=20,
        batch_size=32,
        validation_data=(X_val,y_val))

### **3.a Model Evaluation**

In [ ]:
# Plot Training Results

history_1_df = pd.DataFrame(history_1.history)

plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
sns.lineplot(data=history_1_df[['loss', 'val_loss']])
plt.grid()
plt.title('Loss vs Val-Loss')

plt.subplot(1, 2, 2)
sns.lineplot(data=history_1_df[['accuracy', 'val_accuracy']])
plt.grid()
plt.title('Accuracy vs Val-Accuracy')
plt.show()

In [ ]:
# Model Evaluation Utilities

# fungsi evaluasi
def performance_check(clf, X, y):
  y_pred = clf.predict(X)
  # print('Accuracy Score - : ', accuracy_score(y, y_pred)) # metric Accuracy score
  # print('Precision Score - : ', precision_score(y, y_pred)) # metric precision score
  print('Recall Score - : ', recall_score(y, y_pred)) # metric recall score
  # print('F1 Score - : ', f1_score(y, y_pred)) # metric f1 score
  print('Jumlah class - : ' , Counter(y_pred),'\n')
  cm = confusion_matrix(y, y_pred)
  disp = ConfusionMatrixDisplay(confusion_matrix=cm)
  disp.plot()
  plt.show()
  print(classification_report(y, y_pred))

In [ ]:
result= ann.evaluate(X_train,y_train)
print('loss',result[0])
print('Acc',result[1])

In [ ]:
result= ann.evaluate(X_val,y_val)
print('loss',result[0])
print('Acc',result[1])

In [ ]:
result= ann.evaluate(X_test,y_test)
print('loss',result[0])
print('Acc',result[1])

In [ ]:
# binnary
y_test_proba= ann.predict(X_test)
y_test_pred= tf.where(y_test_proba >= 0.5, 1, 0)

In [ ]:
print(classification_report(y_test, y_test_pred))

In [ ]:
# multi
# Probability for each class

y_pred_df = pd.DataFrame(y_pred, columns=class_names)
y_pred_df

In [ ]:
# multi
# Probability for each class

y_pred_df = pd.DataFrame(y_pred, columns=class_names)
y_pred_df

In [ ]:
# multi
# Get class prediction

y_pred_class = np.argmax(y_pred[0])
y_pred_class_name = class_names[np.argmax(y_pred[0])]

print('Prediction - Class       : ', y_pred_class)
print('Prediction - Class Name  : ', y_pred_class_name)

In [ ]:
# multi
# Check the performance of test-set

## Get the probability
y_pred_mnist_proba = model_sequential_mnist.predict(X_test)

## Get class with maximum probability
y_pred_mnist = np.argmax(y_pred_mnist_proba, axis=-1)

In [ ]:
# multi
# Check the performance of test-set

## Get the probability
y_pred_mnist_proba = model_sequential_mnist.predict(X_test)

## Get class with maximum probability
y_pred_mnist = np.argmax(y_pred_mnist_proba, axis=-1)

## Display Classification Report
print(classification_report(y_test, y_pred_mnist, target_names=np.array(class_names)))

## B. Model Functional

### **1.b Model Definition**

In [ ]:
# Clear Session
seed = 20
tf.keras.backend.clear_session()
np.random.seed(seed)
tf.random.set_seed(seed)

#input layer
input_layer= tf.keras.layers.Input(shape=X_train.shape[1:])

#hidden layer di cabang pertama
dense_1= Dense(300, activation='relu',
               kernel_initializer=tf.keras.initializers.HeNormal(seed),
               name='hidden_1_1')(input_layer)
dense_2= Dense(100, activation='relu',
               kernel_initializer=tf.keras.initializers.HeNormal(seed),
               name='hidden_1_2')(dense_1)

#hidden layer di cabang kedua
dense_3= Dense(300, activation='relu',
               kernel_initializer=tf.keras.initializers.HeNormal(seed),
               name='hidden_2_1')(input_layer)
dense_4= Dense(100, activation='relu',
               kernel_initializer=tf.keras.initializers.HeNormal(seed),
               name='hidden_2_2')(dense_3)

# concat cabang pertama dan kedua
layer_concat= tf.keras.layers.Concatenate()([dense_2,dense_4])

#output layer
output_layer= Dense(1, activation='sigmoid',kernel_initializer=tf.keras.initializers.GlorotNormal(seed))(layer_concat)

#buat model
model_func= Model(inputs=input_layer, outputs=output_layer)

# Compile
model_func.compile(loss='binary_crossentropy',
                         optimizer='adam',
                         metrics=['accuracy'])
model_func.summary()

In [ ]:
# multi class (img)
# Clear Session
seed = 20
tf.keras.backend.clear_session()
np.random.seed(seed)
tf.random.set_seed(seed)

#input layer
input_layer= tf.keras.layers.Input(shape=[28, 28])
flatten_layer = Flatten()(input_layer)

#hidden layer di cabang pertama
dense_1= Dense(16384, activation='relu',
               kernel_initializer=tf.keras.initializers.HeNormal(seed),
               name='hidden_1_1')(flatten_layer)

dense_2= Dense(512, activation='relu',
               kernel_initializer=tf.keras.initializers.HeNormal(seed),
               name='hidden_1_2')(dense_1)

dense_3= Dense(784, activation='relu',
               kernel_initializer=tf.keras.initializers.HeNormal(seed),
               name='hidden_2_3')(dense_2)

reshape_layer = tf.keras.layers.Reshape((28,28))(dense_3)


# concat cabang pertama dan kedua
layer_concat= tf.keras.layers.Concatenate()([input_layer,reshape_layer])

flatten_layer2 = Flatten()(layer_concat)

#output layer
output_layer= Dense(10, activation='softmax')(flatten_layer2)

#buat model
model_func= Model(inputs=input_layer, outputs=output_layer)

# Compile
model_func.compile(loss='sparse_categorical_crossentropy',
                         optimizer='adam',
                         metrics=['accuracy'])
model_func.summary()

In [ ]:
# Plot Layers

tf.keras.utils.plot_model(model_func, show_shapes=True)

In [ ]:
# Plot Layers

tf.keras.utils.plot_model(model_func, show_shapes=True)

### **2.b Model Training**

In [ ]:
history_2= model_func.fit(X_train, y_train,
        epochs=20,
        batch_size=32,
        validation_data=(X_val,y_val))

In [ ]:
# multi class 
# Train the Model

%%time
history_model_func = model_func.fit(X_train, y_train, epochs=30, validation_data=(X_val, y_val))

### **3.b Model Evaluation**

In [ ]:
y_test_proba= model_func.predict(X_test)
y_test_pred= tf.where(y_test_proba >= 0.5, 1, 0)

In [ ]:
print(classification_report(y_test,y_test_pred))

---

# Model Inference

In [ ]:
import pickle

# Save the preprocessing to a file
with open('preprocessing.pkl', 'wb') as file:
    pickle.dump(final_pipeline, file)

# Load the preprocessing from the file
with open('preprocessing.pkl', 'rb') as file:
    preproc_final = pickle.load(file)

#save model
ann.save('model_ann.keras')

# load model
model_final= load_model('model_ann.keras')

In [ ]:
data_inf= data.sample(5)

In [ ]:
data_inf.drop(['PassengerId', 'Cabin', 'Ticket', 'Embarked', 'Name'], axis=1, inplace=True)

In [ ]:
data_inf.head()

In [ ]:
data_inf_preproc= preproc_final.transform(data_inf)

In [ ]:
pred_proba=model_final.predict(data_inf_preproc)
pred= tf.where(pred_proba >=0.5,1,0)

In [ ]:
data_inf['predict']= pred
data_inf

---

# vii. Kesimpulan